In [26]:
!pip install loguru
import os
import sys

# Create the utils folder and logger.py file
os.makedirs("utils", exist_ok=True)
with open("utils/logger.py", "w") as f:
    f.write("from loguru import logger\n")

# Add current directory to Python path
sys.path.append(os.path.abspath("."))

# Now safely import logger
from utils.logger import logger

logger.info("✅ Logger is working and utils/logger.py is found!")





2025-07-12 20:21:35.642 | INFO     | __main__:<module>:16 - ✅ Logger is working and utils/logger.py is found!


In [34]:
# Install required packages (run once in your notebook or terminal)
!pip install --upgrade pip
!pip install telethon python-dotenv loguru

# Imports
import os
import json
import asyncio
from datetime import datetime
from telethon import TelegramClient
from dotenv import load_dotenv
from loguru import logger

# Load environment variables
load_dotenv()

# Validate and get environment variables
api_id_raw = os.getenv("API_ID")
if api_id_raw is None:
    raise ValueError("Missing API_ID from .env file")
API_ID = int(api_id_raw)

API_HASH = os.getenv("API_HASH")
if API_HASH is None:
    raise ValueError("Missing API_HASH from .env file")

SESSION_NAME = os.getenv("SESSION_NAME")
if SESSION_NAME is None:
    raise ValueError("Missing SESSION_NAME from .env file")

# Telegram channels to scrape
CHANNELS = [
    "https://t.me/CheMed123",
    "https://t.me/lobelia4cosmetics",
    "https://t.me/tikvahpharma"
]

# Prepare folders to store raw data and images
TODAY = datetime.now().strftime("%Y-%m-%d")
BASE_PATH = f"data/raw/telegram_messages/{TODAY}"
IMAGE_PATH = f"data/raw/images/{TODAY}"
os.makedirs(BASE_PATH, exist_ok=True)
os.makedirs(IMAGE_PATH, exist_ok=True)

# Initialize Telegram client
client = TelegramClient(SESSION_NAME, API_ID, API_HASH)

async def scrape_channel(channel_url):
    channel_name = channel_url.split("/")[-1]
    logger.info(f"Scraping channel: {channel_name}")
    messages_data = []
    channel_img_path = os.path.join(IMAGE_PATH, channel_name)
    os.makedirs(channel_img_path, exist_ok=True)

    try:
        async for message in client.iter_messages(channel_url, limit=100):
            msg = {
                "id": message.id,
                "date": str(message.date),
                "sender_id": message.sender_id,
                "text": message.text,
                "has_photo": bool(message.photo),
                "channel": channel_name
            }

            if message.photo:
                image_name = f"{message.id}.jpg"
                image_path = os.path.join(channel_img_path, image_name)
                try:
                    await client.download_media(message.photo, file=image_path)
                    msg["image_path"] = image_path
                    logger.debug(f"Downloaded image: {image_path}")
                except Exception as e:
                    logger.warning(f"Image download failed: {e}")
                    msg["image_path"] = None

            messages_data.append(msg)

        # Save messages JSON file
        out_file = os.path.join(BASE_PATH, f"{channel_name}.json")
        with open(out_file, "w", encoding="utf-8") as f:
            json.dump(messages_data, f, ensure_ascii=False, indent=2)

        logger.success(f"Scraped and saved {len(messages_data)} messages from {channel_name}")

    except Exception as e:
        logger.error(f"Error scraping {channel_url}: {e}")

async def main():
    await client.start()
    tasks = [scrape_channel(url.strip()) for url in CHANNELS if url.strip()]
    await asyncio.gather(*tasks)
    await client.disconnect()

if __name__ == "__main__":
    asyncio.run(main())


ValueError: Missing API_ID from .env file